In [17]:
import os
import requests
from flask import Flask, render_template_string, request

app = Flask(__name__)

def get_real_time_temperature(city, country, api_key):
    """Use the free Weather API endpoint instead of OneCall API"""
    base_url = "api.openweathermap.org/data/2.5/weather?q=London,uk&APPID=51954a19a7127c68dabe9f25100b228a"
    params = {
        'q': f"{city},{country}",
        'appid': api_key,
        'units': 'metric'
    }
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        weather_data = response.json()
        temperature = weather_data['main']['temp']
        return temperature, None
    except requests.exceptions.HTTPError as http_err:
        return None, f"HTTP error occurred: {http_err}"
    except Exception as err:
        return None, f"Error getting temperature: {err}"

@app.route('/', methods=['GET', 'POST'])
def index():
    temperature = None
    error_message = None

    if request.method == 'POST':
        city = request.form.get('city')
        country = request.form.get('country')
        api_key = os.getenv('51954a19a7127c68dabe9f25100b228a')

        if not api_key:
            error_message = "API key not found. Please set the WEATHER_API_KEY environment variable."
        elif city and country:
            temperature, error = get_real_time_temperature(city, country, api_key)
            if error:
                error_message = error
    
    # ...rest of the template code remains the same...

    return render_template_string('''
        <!doctype html>
        <html lang="en">
          <head>
            <meta charset="utf-8">
            <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
            <title>Weather App</title>
            <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css">
          </head>
          <body>
            <div class="container">
              <h1 class="mt-5">Real-Time Temperature Checker</h1>
              <form method="post" class="mt-3">
                <div class="form-group">
                  <label for="city">City</label>
                  <input type="text" class="form-control" id="city" name="city" required>
                </div>
                <div class="form-group">
                  <label for="country">Country</label>
                  <input type="text" class="form-control" id="country" name="country" required>
                </div>
                <button type="submit" class="btn btn-primary">Get Temperature</button>
              </form>
              {% if temperature is not none %}
                <div class="alert alert-info mt-3">
                  The current temperature in {{ request.form.city }}, {{ request.form.country }} is {{ temperature }}°C.
                </div>
              {% elif error_message %}
                <div class="alert alert-danger mt-3">
                  {{ error_message }}
                </div>
              {% endif %}
            </div>
            <script src="https://code.jquery.com/jquery-3.3.1.slim.min.js"></script>
            <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.7/umd/popper.min.js"></script>
            <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/js/bootstrap.min.js"></script>
          </body>
        </html>
        ''', temperature=temperature, error_message=error_message)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Mar/2025 10:15:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2025 10:15:17] "POST / HTTP/1.1" 200 -
